## DEMO: Finding all NC highway segments within 1500 m of a restaurant
This script demonstrates how Python can fetch data from OpenStreetMap to reveal which sections of major roads in North Carolina occur within 1.5 km of a restaurant. 

The workflow for this analysis is as follows:
* Fetch NC highways as a graph (save as GraphML) 
 * Create a spatial dataframe of North Carolina using `osmnx`'s `gdf_from_place()` function
 * Extract the geometric shape from this geodataframe, used as the boundary to query OSM data.
 * Fetch "motorways" found within this shape.
* Convert the result (a graph object) into a geodataframe
 * Save the geodataframe as a shapefile (for later use)
* Buffer road features 1500m 
 * Extract the geometries as a geoseries object
 * Applt the `to_crs()` function to tranform the coordinate reference system from WGS 84 to UTM Zone 17N
 * Apply the `buffer` command to the buffer 

In [ ]:
#Import osmnx package
import osmnx as ox
import geopandas as gpd

In [ ]:
#Create a geodataframe of triangle counties
aoi_gdf = ox.gdf_from_places(['Durham County, NC',
                              'Chatham County, NC',
                              'Orange County, NC',
                              'Wake County, NC'])

In [ ]:
aoi_gdf.plot('place_name');

In [ ]:
#Dissolve all shapes into one
aoi_poly = aoi_gdf.geometry.unary_union
aoi_poly

In [ ]:
#Fetch highways in within the polygon
hwy_graph = ox.graph_from_polygon(aoi_poly,  
                                  network_type='drive',
                                  simplify=True,
                                  retain_all=False,
                                  truncate_by_edge=True,
                                  infrastructure='way["highway"~"motorway"]')

In [ ]:
#Convert graph to a pair of geodataframe (edges and nodes)
gdf_nodes, gdf_edges = ox.graph_to_gdfs(hwy_graph)

In [ ]:
#Plot
ax = aoi_gdf.plot(alpha=0.4,figsize=(10,7))
gdf_edges.plot(ax = ax, color='red');

In [ ]:
#Project the dataset to UTM Zone 17N
gdf_edges_utm = gdf_edges.to_crs(epsg=32617)
gdf_edges_utm.plot();

In [ ]:
#Buffer, transform, and dissolve the roads
hwy_buffer = (gdf_edges_utm.geometry
              .buffer(1500)
              .to_crs(4326)
              .unary_union)

In [ ]:
#Plot
hwy_buffer

In [ ]:
#Fetch restaurants falling within the road buffer
nc_food = ox.pois.create_poi_gdf(polygon=hwy_buffer,amenities=['restaurant'])

In [ ]:
import contextily as ctx
ax = gdf_edges_utm.to_crs(epsg=3857).plot(alpha=0.1,figsize=(10,10))
nc_food.geometry.to_crs(epsg=3857).plot(ax = ax,color='red')
ctx.add_basemap(ax)

In [ ]:
#Buffer restaurants
nc_food_1500m = (nc_food.to_crs(epsg=32617)
                 .buffer(1500)
                 .to_crs(4326))
#Plot
nc_food_1500m.plot();

In [ ]:
gdf_Food = gpd.GeoDataFrame(geometry=nc_food_1500m)

In [ ]:
#Clip roads
roads_clip = gpd.overlay(gdf_edges,gdf_Food,how='intersection')

In [ ]:
ax = roads_clip.plot(figsize=(20,20))
nc_food.plot(ax=ax,color='red',);